In [10]:
"""
This script computes the optimal SE(3) transformation between corresponding 2D and 3D points using non-linear optimiziation.
"""
import numpy as np
from numpy.typing import NDArray
import open3d as o3d
import scipy

In [11]:
# Define the 3D points in world coordinates
points_3d = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 1]])

# Define the 2D points in image coordinates
points_2d = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

# Define the camera intrinsic matrix
K = np.array([[1.0, 0.0, 0.5], [0.0, 1.0, 0.5], [0.0, 0.0, 1.0]])

# Define the initial camera extrinsic parameters
R = np.eye(3)
t = np.zeros(3)

In [12]:
K = np.eye(3)

def project(R: NDArray, t: NDArray, K: NDArray, points_3d: NDArray) -> NDArray:
    """
    This function projects 3D points onto the image plane.
    """
    # Compute the projection matrix
    P = K @ np.hstack([R, t.reshape(-1, 1)])
    
    # Convert the 3D points to homogeneous coordinates
    points_3d_h = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    
    # Project the 3D points onto the image plane
    points_2d = P @ points_3d_h.T
    points_2d = points_2d.T
    points_2d = points_2d[:, :2] / points_2d[:, 2:]
    
    return points_2d

# Define the optimization objective function
def f(x: NDArray, points_3d, points_2d, K=K):
    """
    This function computes the difference between the predicted and observed 2D points.
    """
    # Unpack the angle-axis vector and translation vector
    theta = x[0]
    axis = x[1:3]
    t = x[3:]
    
    # Compute the rotation matrix
    R = o3d.geometry.get_rotation_matrix_from_axis_angle(theta*axis)
    
    # Project the 3D points onto the image plane
    points_2d_pred = project(R, t, K, points_3d)
    
    # Compute the difference between the predicted and observed 2D points
    diff = points_2d_pred - points_2d
    diff = diff.reshape(-1)
    return diff

In [13]:
# Define the constraint functions
def g1(x: NDArray):
    """
    This function computes the constraint that the angle-axis vector must have unit norm.
    """
    return np.linalg.norm(x[1:3]) - 1

def g2(x: NDArray):
    """
    This function computes the constraint that the translation vector must have unit norm.
    """
    return np.linalg.norm(x[3:]) - 1

# Define the angle constraint
def g3(x: NDArray):
    """
    This function computes the constraint that the angle must be between 0 and pi/2.
    """
    return x[0] - np.pi/2

In [ ]:
# Define KKT conditions
def kkt(x: NDArray, points_3d, points_2d, K=K):
    """
    This function computes the KKT conditions.
    """
    # Compute the Jacobian of the objective function
    J = scipy.optimize.approx_fprime(x, f, 1e-8, points_3d, points_2d, K)
    
    # Compute the Jacobian of the constraint functions
    J1 = scipy.optimize.approx_fprime(x, g1, 1e-8)
    J2 = scipy.optimize.approx_fprime(x, g2, 1e-8)
    J3 = scipy.optimize.approx_fprime(x, g3, 1e-8)
    
    # Compute the KKT conditions
    kkt = np.hstack([J, J1, J2, J3])
    return kkt

In [16]:
# Define the initial guess
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# Define the bounds
bnds = ((-np.inf, np.inf), (-1, 1), (-1, 1), (-np.inf, np.inf), (-np.inf, np.inf))

# Define the constraints
cons = ({'type': 'eq', 'fun': g1}, {'type': 'eq', 'fun': g2}, {'type': 'ineq', 'fun': g3})

# Perform the optimization
# res = scipy.optimize.minimize(f, x0, args=(points_3d, points_2d), 
#     # method='SLSQP',
#     method='lm',
#     bounds=bnds, constraints=cons)

res = run_optimization(f, x0, args=(points_3d, points_2d))

# Print the results
print(res)

# Unpack the angle-axis vector and translation vector
theta = res.x[0]
axis = res.x[1:3]
t = res.x[3:]

# Compute the rotation matrix
R = o3d.geometry.get_rotation_matrix_from_axis_angle(theta*axis)

# Print the rotation matrix
print(R)

# Print the translation vector
print(t)

# Project the 3D points onto the image plane
points_2d_pred = project(R, t, K, points_3d)

# Print the predicted 2D points
print(points_2d_pred)

# Print the observed 2D points
print(points_2d)

# Plot the 3D points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_3d)
o3d.visualization.draw_geometries([pcd])

# Plot the 2D points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_2d)
o3d.visualization.draw_geometries([pcd])

# Plot the predicted 2D points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_2d_pred)
o3d.visualization.draw_geometries([pcd])



NameError: name 'run_optimization' is not defined